In [1]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

This notebook computes the latencies and bandwidths of the three primitive function types

In [2]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
        }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))
pd.set_option('precision',1)

#### Axpby and Dot Latencies
The latencies are determined by taking the minimum of the average runtimes 

#### Axpby and Dot Bandwidths
The bandwidths are determined by taking the average bandwidth of the 30 bandwidths corresponding to the 3 largest sizes.

#### Dx-Dy Latencies
As in Axpby 

#### Dx-Dy Bandwidths
Since the efficiency of the matrix-vector multiplications depends on the polynomial coefficient we should compute these bandwidths separately


In [70]:
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
#ns=[3,4]
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
    runtimes=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    runtimes.insert(0,'size', 8*runtimes['n']*runtimes['n']*runtimes['Nx']*runtimes['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        runtimes.insert(0,name+'_bw',runtimes['size']/1000*memops/runtimes[name])
    runtimes = runtimes.assign( dxdy=(runtimes['dx']+runtimes['dy'])/2)
    runtimes = runtimes.assign( dxdy_bw=2.0*runtimes['dx_bw']*runtimes['dy_bw']/(runtimes['dx_bw']+runtimes['dy_bw']))
    #compute one version with aggregated grouped sizes and one without
    avgruntimes=runtimes.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    avgruntimes=avgruntimes.reset_index(level=['n','Nx','Ny','size'])
    avgruntimes.sort_values(by='size',inplace=True) #sort by size
    runtimes.sort_values(by='size',inplace=True)
    ##first compute axpby and dot latencies and bandwidths 
    nmax = 3
    s =30

    line = []
    l=len(runtimes)
    line.append(v[0]) #0
    line.append(v[1]) #1
    line.append(runtimes[l-s:l]['axpby_bw'].mean()) #2
    line.append(runtimes[l-s:l]['axpby_bw'].std())  #3
    line.append(avgruntimes[0:nmax][('axpby','mean')].mean()/1e-6) #4
    line.append(avgruntimes[0:nmax][('axpby','mean')].min()/1e-6)  #5

    line.append( line[5] - avgruntimes['size'][0]*names['axpby']/line[2]/1e-3) #6
    if line[6] <0 : line[6] = 0 

    line.append(runtimes[l-s:l]['dot_bw'].mean()) #7
    line.append(runtimes[l-s:l]['dot_bw'].std())  #8 
    line.append(avgruntimes[0:nmax][('dot','mean')].mean()/1e-6) #9
    line.append(avgruntimes[0:nmax][('dot','mean')].min()/1e-6)  #10
    line.append(line[10] - avgruntimes['size'][0]*names['dot']/line[2]/1e-3) #11
    if line[11] <0 : line[11] = 0 
    ##now compute latency and bandwidths of dx and y
 
    for n in [2,3,4,5]:
        #take n
        dxdy=runtimes[runtimes['n']==n]
        
        avgdxdy = avgruntimes[avgruntimes['n']==n]
        dxdy=dxdy.sort_values(by='size')
        avgdxdy=avgdxdy.sort_values(by='size') #sort by size

        mb=9
        Nx = 767 # compute among the four greates sizes
        line.append(dxdy[dxdy['Nx']>=Nx]['dxdy_bw'].mean()) #12
        line.append(dxdy[dxdy['Nx']>=Nx]['dxdy_bw'].std())  #13   
        line.append(avgdxdy[0:nmax][('dxdy','mean')].mean()/1e-6) #14
        line.append(avgdxdy[('dxdy','mean')].min()/1e-6)     #15
        line.append(avgdxdy[('dxdy','mean')].min()/1e-6 - avgdxdy['size'].iloc[0]*names['dx']/line[2]/1e-3)#16
        if line[len(line)-1] <0 : line[len(line)-1] = 0        
    #print(line)    
    values.append(line)

In [72]:
#now construct new table with values from previous cell      
tuples=[('arch','',''),('nodes','','')]        
for q in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append((q,'bw','avg'))
    tuples.append((q,'bw','std'))
    tuples.append((q,'lat','avg'))
    tuples.append((q,'lat','min'))
    tuples.append((q,'lat','bw'))

cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
arr

axpby                               dot                         \
                  bw            lat                 bw             lat          
                 avg      std   avg   min    bw    avg      std    avg    min   
arch    nodes                                                                   
gtx1060 1      157.0  5.6e-02  23.2   3.5   0.0   26.5  9.8e-02  199.9  131.6   
i5      1       30.0  1.9e-01  30.6  12.4   0.0    9.3  3.7e-02  316.7  117.4   
knl     1      442.6  1.1e+01  15.4  13.4   9.4  143.6  8.4e+00   80.2   68.7   
        2      449.6  1.5e+01  28.5  14.7  12.7  131.6  4.8e+00  103.8   96.5   
        4      426.2  3.9e+01  26.0  19.0  18.0  113.3  7.2e+00  134.0  132.1   
p100    1      552.8  1.0e+00   4.1   3.1   0.2  345.8  1.9e+00   61.1   56.4   
        2      554.2  1.2e-01   3.5   3.3   1.9  338.2  3.2e+00   60.3   56.1   
        4      554.8  1.7e+00   3.2   3.1   2.4  320.3  7.5e+00   72.2   68.0   
skl     1      206.8  3.0e+00   6.6   6.1   0.0  193.6  8.6e+00   31.4   24.5   
        2      210.4  6.1e+00   8.1   6.9   3.1  184.5  8.4e+00   34.1   32.6   
        4      225.5  1.3e+01   8.5   6.0   4.3  173.0  1.2e+01   42.5   40.7   
v100    1      848.3  6.8e-01   3.8   3.1   1.2  591.6  2.9e+00   42.6   41.5   
        2      847.9  3.7e-01   3.3   3.1   2.2  575.3  7.0e+00   50.0   48.2   
        4      844.9  1.8e+00   3.4   3.4   2.9  525.7  1.4e+01   62.5   59.2   

                      ...    dxdy4                               dxdy5        \
                      ...       bw            lat                   bw         
                  bw  ...      avg    std     avg    min     bw    avg   std   
arch    nodes         ...                                                      
gtx1060 1      124.9  ...     80.0   15.6   322.6   72.5   32.4   61.1  18.8   
i5      1       82.5  ...     25.8    1.0  1212.1  208.3    0.0   20.8   1.4   
knl     1       66.0  ...    118.7    8.7   225.9   58.4   42.4   95.9   3.1   
        2       95.2  ...    107.4    4.4   303.2  212.7  204.8   88.4   4.7   
        4      131.4  ...     88.0   12.4   288.8  216.8  212.6   75.2   6.9   
p100    1       54.5  ...    200.7    1.7   148.6   30.1   18.8  162.6  18.1   
        2       55.2  ...    146.0   27.9   543.7  484.8  479.1  140.2  17.6   
        4       67.5  ...    102.0   35.0   728.4  699.1  696.2  108.6  27.5   
skl     1       19.4  ...    116.4    3.9   374.2   85.6   55.2   96.7   9.4   
        2       30.1  ...    104.1   17.5   167.6   78.3   63.3  100.8  11.0   
        4       39.5  ...     93.4   15.3   102.9   64.6   57.6   94.3  17.0   
v100    1       40.3  ...    656.3   11.6    47.1   12.6    5.1  522.9  60.7   
        2       47.6  ...    332.8  117.0   472.6  452.2  448.5  358.2  94.6   
        4       58.9  ...    176.2   90.2   711.5  695.6  693.7  216.5  95.8   

                                     
                  lat                
                  avg    min     bw  
arch    nodes                        
gtx1060 1       571.4  125.0   62.5  
i5      1      1956.6  340.4   12.6  
knl     1       460.3   92.7   67.6  
        2       480.2  342.2  329.9  
        4       382.1  291.5  284.9  
p100    1       273.0   62.7   45.0  
        2       606.3  491.4  482.5  
        4       763.3  694.4  690.0  
skl     1       423.0  132.0   84.4  
        2       254.1  101.1   77.8  
        4       149.4   73.8   62.9  
v100    1        85.5   23.6   12.1  
        2       492.7  454.8  449.0  
        4       721.0  688.0  685.1  

[14 rows x 30 columns]

In [73]:
#arr=arr.reset_index()

In [74]:
#define conversion function 
def toString(x): 
    np.ceil(x)
    #string = '%.1f'% x
    string = '%d' %np.ceil(x)
    return string

In [76]:
addto = []
for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    addto.append((n,'lat','bw'))
    addto.append((n,'bw','string'))

#make a table for display
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
newindex.append(('gtx1060',1))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable=nicetable.reindex(newindex)
nicetable

axpby              dot            dxdy2            dxdy3  \
                lat        bw    lat        bw    lat        bw    lat   
arch    nodes                                                            
i5      1       0.0    30 ± 1   82.5    10 ± 1    0.0    28 ± 3    0.0   
skl     1       0.0   207 ± 3   19.4   194 ± 9   33.5  153 ± 40   33.5   
        2       3.1   211 ± 7   30.1   185 ± 9   44.3  142 ± 33   52.1   
        4       4.3  226 ± 13   39.5  173 ± 13   43.6  114 ± 17   47.3   
knl     1       9.4  443 ± 12   66.0   144 ± 9   19.2  249 ± 12   25.2   
        2      12.7  450 ± 15   95.2   132 ± 5   92.8  154 ± 33  137.0   
        4      18.0  427 ± 40  131.4   114 ± 8  141.0  102 ± 27  176.2   
gtx1060 1       0.0   158 ± 1  124.9    27 ± 1    1.7   131 ± 1   11.9   
p100    1       0.2   553 ± 2   54.5   346 ± 2    4.5   288 ± 3    9.3   
        2       1.9   555 ± 1   55.2   339 ± 4  463.6  108 ± 48  469.9   
        4       2.4   555 ± 2   67.5   321 ± 8  681.3   53 ± 31  676.4   
v100    1       1.2   849 ± 1   40.3   592 ± 3    3.3  802 ± 17    3.6   
        2       2.2   848 ± 1   47.6   576 ± 8  451.6  160 ± 91  452.9   
        4       2.9   845 ± 2   58.9  526 ± 14  690.7   62 ± 40  690.4   

                          dxdy4             dxdy5            
                      bw    lat         bw    lat        bw  
arch    nodes                                                
i5      1         29 ± 1    0.0     26 ± 1   12.6    21 ± 2  
skl     1       152 ± 13   55.2    117 ± 4   84.4   97 ± 10  
        2       134 ± 28   63.3   105 ± 18   77.8  101 ± 11  
        4       131 ± 24   57.6    94 ± 16   62.9   95 ± 17  
knl     1       170 ± 23   42.4    119 ± 9   67.6    96 ± 4  
        2       125 ± 10  204.8    108 ± 5  329.9    89 ± 5  
        4       103 ± 25  212.6    89 ± 13  284.9    76 ± 7  
gtx1060 1        111 ± 1   32.4    80 ± 16   62.5   62 ± 19  
p100    1        240 ± 2   18.8    201 ± 2   45.0  163 ± 19  
        2       140 ± 41  479.1   146 ± 28  482.5  141 ± 18  
        4        85 ± 39  696.2   102 ± 35  690.0  109 ± 28  
v100    1       723 ± 10    5.1   657 ± 12   12.1  523 ± 61  
        2      262 ± 121  448.5  333 ± 118  449.0  359 ± 95  
        4       120 ± 71  693.7   177 ± 91  685.1  217 ± 96

#### Assumptions
- there are three basic functions: trivially parallel(axpby), nearest neighbor (dxdy), global reduction (dot)
- each can be represented by the single node bandwidth, the single node latency and the multinode latency

#### But
- does not capture cache effect e.g. in SKl

In [77]:
index = ['i5','skl','knl','gtx1060','p100','v100']  
lines = []
for arch in  index: 
    line = []
    line.append(arch)
    #first the bandwidths
    line.append( arr.loc[(arch,1),('axpby','bw','avg')] )
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        line.append( arr.loc[(arch,1),(n,'bw','avg')] /line[1])
    for n in ['axpby','dot','dxdy2'] :
        line.append( arr.loc[(arch,1),(n,'lat','bw')] )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(None)
        else:
            line.append( arr.loc[(arch,4),(n,'lat','bw')] )
    lines.append(line)
    
tuples=['arch']     

for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append(n+'_bw')
for n in ['axpby','dot','dxdy']:
    tuples.append(n+'_lat_shared')
    tuples.append(n+'_lat_dist')
cols=tuples
toDisk = pd.DataFrame(lines, columns=cols)
toDisk.to_csv('performance.csv',sep=' ',index=False)

In [78]:
test = pd.read_csv('performance.csv',delimiter=' ')
test

,arch,axpby_bw,dot_bw,dxdy2_bw,dxdy3_bw,dxdy4_bw,dxdy5_bw,axpby_lat_shared,axpby_lat_dist,dot_lat_shared,dot_lat_dist,dxdy_lat_shared,dxdy_lat_dist
0,i5,30.0,0.3,0.9,1.0,0.9,0.7,0.0,NaN,82.5,NaN,0.0,NaN
1,skl,206.8,0.9,0.7,0.7,0.6,0.5,0.0,4.3,19.4,39.5,33.5,43.6
2,knl,442.6,0.3,0.6,0.4,0.3,0.2,9.4,18.0,66.0,131.4,19.2,141.0
3,gtx1060,157.0,0.2,0.8,0.7,0.5,0.4,0.0,NaN,124.9,NaN,1.7,NaN
4,p100,552.8,0.6,0.5,0.4,0.4,0.3,0.2,2.4,54.5,67.5,4.5,681.3
5,v100,848.3,0.7,0.9,0.9,0.8,0.6,1.2,2.9,40.3,58.9,3.3,690.7
